# Prepping for real run. 

Todo: 
* Define `max_source_length` and `max_target_length` for the model (otherwise truncated).
padding token should be replaced with -100, which is the 'ignore_index' of `CrossEntorpyLoss` in PT and TF. For Flax, use `decoder_attention_mask`. 
Attention_mask. ensures madding tokens of inputs are ignored. 

* Install apex. "model will automatically use apex.normalization.FusedRMSNorm instead of T5LayerNorm." The former uses an optimized fused kernel which is several times faster than the latter.

A note on model sizes: 
T5-11B (original, not v1.1) weights in float32 are 45.2GB. 
See this post for using huggingface endpoints on SINGLE GPU for cheap inference: https://www.philschmid.de/deploy-t5-11b
Uses mixed precision and sharding, and LLM.int8(). 

In [ ]:
import torch
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Model, T5Config, AutoModelWithLMHead
import accelerate
import wandb
# !wandb login  -- reactivate later
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
'''
MODEL SELECTION

T5 V1.1 --  https://huggingface.co/docs/transformers/model_doc/t5v1.1 && https://github.com/google-research/text-to-text-transfer-transformer/blob/main/released_checkpoints.md#t511
small - base - large - 3b/xl - 11b/xxl

OG: t5-small

'google/t5-base-lm-adapt' # largest on my server (without float16)
'google/t5-xl-lm-adapt'
'''

# MODEL_SIZE = "t5-base"
MODEL_NAME = "google/t5-small-lm-adapt"
# config = T5Config.from_pretrained(MODEL_NAME)
t5 = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, low_cpu_mem_usage=True).to(device)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME, return_special_tokens_mask=True)
# low_cpu_mem_usage(bool, optional) — Tries to not use more than 1x model size in CPU memory (including peak memory) while loading the model. experimental.

In [ ]:
''' MODEL QUANTIZATION: fp16 '''

# XL worked on my server (tested both CPU and GPU).
# MODEL_NAME = "google/t5-base-lm-adapt"
# t5 = AutoModelWithLMHead.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, low_cpu_mem_usage=True).to(device)
# tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME, return_special_tokens_mask=True)
# low_cpu_mem_usage(bool, optional) — Tries to not use more than 1x model size in CPU memory (including peak memory) while loading the model. experimental.

''' int8 -- my GPU doesn't support it.. '''
# t5 = AutoModelWithLMHead.from_pretrained(MODEL_NAME, load_in_8bit=True, device_map='auto', low_cpu_mem_usage=True)

# Moc Dataset

In [ ]:
''' PREP EMBEDDING INPUTS '''
# shape = (batch_size, 'words', embedding_dim) -- here 'words' == each of our embeddings, like clip and language.
# one_input_shape = [6, 512, 512]
one_input_shape = [1, 512, 512]

decoder_input_embeds_arr = np.random.rand( *one_input_shape ).astype(np.float32) # need fp32
decoder_input_embeds_arr = torch.from_numpy(decoder_input_embeds_arr).to(device)
input_embeds_arr = np.random.rand( *one_input_shape ).astype(np.float32)
input_embeds_arr = torch.from_numpy(input_embeds_arr).to(device)
attn_mask_arr = np.ones( one_input_shape )
attn_mask_arr = torch.from_numpy(attn_mask_arr).to(device)

print(decoder_input_embeds_arr.shape)
print(input_embeds_arr.shape)
print(attn_mask_arr.shape)

''' Decoder gets the tokenized caption. Shape is (batch_size, max_caption_length). Use padding to make it fit. '''
# WORKING example, but easier with numpy.
# import torch.nn.functional as F
# decoder_input_ids = tokenizer("This is the target output sentence, aka the video caption. I like tacos because they are so delicious.", return_tensors="pt").input_ids.to(device)
# decoder_input_ids = F.pad(decoder_input_ids, (0, (512-decoder_input_ids.shape[1])), value=tokenizer.pad_token_id)
# print(decoder_input_ids.shape)
# decoder_input_ids
decoder_input_ids = torch.from_numpy(np.random.randint(1, 10_000, size=(one_input_shape[0], one_input_shape[2]))).to(device)
print(decoder_input_ids.shape)

## Train function

T5 forward() docs: https://huggingface.co/docs/transformers/model_doc/t5#transformers.T5ForConditionalGeneration.forward

Todo: investigate difference between decoder `decoder_input_ids` and `lm_labels`.
For example: 
```
outputs = t5(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
```

I think `loss.sum()` is for multi-iteration loss. I was inadverdently using it 6 batches.
https://discuss.pytorch.org/t/loss-backward-raises-error-grad-can-be-implicitly-created-only-for-scalar-outputs/12152 
loss.backward() # T5 RuntimeError: grad can be implicitly created only for scalar outputs

In [ ]:
t5.train()
# outputs = t5.forward(inputs_embeds=input_embeds_arr, attention_mask=attn_mask_arr, decoder_inputs_embeds=input_embeds_arr)
outputs = t5.forward(inputs_embeds=input_embeds_arr, attention_mask=attn_mask_arr, decoder_input_ids=decoder_input_ids, decoder_attention_mask=attn_mask_arr)
loss = outputs[0]
loss.shape

In [ ]:
''' backwards pass '''
optimizer = torch.optim.Adam(params =  t5.parameters(), lr=1e-4)
optimizer.zero_grad()
loss.sum().backward()
optimizer.step()
print("✅ Successful training iteration")

In [ ]:
''' minimal train '''
t5.train()

loss = outputs[0]


optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-4)
optimizer.zero_grad()
loss.backward()
optimizer.step()

## Example from 

In [ ]:
# Creating the training function. This will be called in the main function. It is run depending on the epoch value.
# The model is put into train mode and then we enumerate over the training loader and passed to the defined network 

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
        loss = outputs[0]
        
        if _%10 == 0:
            wandb.log({"Training Loss": loss.item()})

        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)  # FOR TPU
        # xm.mark_step()                # FOR TPU

In [ ]:
def main():
    # WandB – Initialize a new run
    wandb.init(project="transformers_tutorials_summarization")

    # WandB – Config is a variable that holds and saves hyperparameters and inputs
    # Defining some key variables that will be used later on in the training  
    config = wandb.config          # Initialize config
    config.TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
    config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
    config.TRAIN_EPOCHS = 2        # number of epochs to train (default: 10)
    config.VAL_EPOCHS = 1 
    config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
    config.SEED = 42               # random seed (default: 42)
    config.MAX_LEN = 512
    config.SUMMARY_LEN = 150 

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(config.SEED) # pytorch random seed
    np.random.seed(config.SEED) # numpy random seed
    torch.backends.cudnn.deterministic = True

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    

    # Importing and Pre-Processing the domain data
    # Selecting the needed columns only. 
    # Adding the summarzie text in front of the text. This is to format the dataset similar to how T5 model was trained for summarization task. 
    df = pd.read_csv('./data/news_summary.csv',encoding='latin-1')
    df = df[['text','ctext']]
    df.ctext = 'summarize: ' + df.ctext
    print(df.head())

    
    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest will be used for validation. 
    train_size = 0.8
    train_dataset=df.sample(frac=train_size,random_state = config.SEED)
    val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    print("FULL Dataset: {}".format(df.shape))
    print("TRAIN Dataset: {}".format(train_dataset.shape))
    print("TEST Dataset: {}".format(val_dataset.shape))


    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
    val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

    # Defining the parameters for creation of dataloaders
    train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

    val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)


    
    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
    model = model.to(device)

    # Defining the optimizer that will be used to tune the weights of the network in the training session. 
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

    # Log metrics with wandb
    wandb.watch(model, log="all")
    # Training loop
    print('Initiating Fine-Tuning for the model on our dataset')

    for epoch in range(config.TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)


    # Validation loop and saving the resulting file with predictions and acutals in a dataframe.
    # Saving the dataframe as predictions.csv
    print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
    for epoch in range(config.VAL_EPOCHS):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
        final_df.to_csv('./models/predictions.csv')
        print('Output Files generated for review')

if __name__ == '__main__':
    main()